In [ ]:
import numpy as np
import cv2
from keras.preprocessing.image import img_to_array
from keras.models import load_model


xml = r'C:\Users\user\haarcascade_frontalface_alt.xml'
hdf5 = r'C:\Users\user\emotion_model.hdf5'

face_cascade = cv2.CascadeClassifier(xml)
emotion_classifier = load_model(hdf5)
EMOTIONS = ["Angry", "Disgusting", "Fearful", "Happy", "Sad", "Surpring", "Neutral"]

cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)

while(True):
    ret, frame = cap.read() # Capture
    frame = cv2.flip(frame,1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Gray로 Convert
    
    faces = face_cascade.detectMultiScale(gray,1.05,5) # 얼굴 정보 찾기
    
    canvas = np.zeros((250,300,3), dtype="uint8") # 빈 이미지 공간 생성
    
    if len(faces) > 0:
        face = sorted(faces, reverse=True, key=lambda x: (x[2]-x[0])*(x[3]-x[1]))[0]
        (fX, fY, fW, fH) = face
        
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi,(48,48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)
        
        predic = emotion_classifier.predict(roi)[0]
        emotion_probability = np.max(predic)
        label = EMOTIONS[predic.argmax()]
        
        cv2.putText(frame, label, (fX, fY-10), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0,0,255), 2)
        cv2.rectangle(frame, (fX, fY), (fX+fW, fY+fH), (0,0,255),2)
        
        for (i, (emotion, prob)) in enumerate(zip(EMOTIONS, predic)):
            text = "{}: {:.2f}%".format(emotion, prob*100)
            w = int(prob*300)
            cv2.rectangle(canvas, (7,(i*35)+5),(w,(i*35)+35),(0,0,255),-1)
            cv2.putText(canvas, text, (10,(i*35)+23), cv2.FONT_HERSHEY_SIMPLEX, 0.45, (255,255,255),2)
     
    cv2.imshow('Your Face', frame)
    cv2.imshow('Calculating...', canvas)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
cap.release()
cv2.destroyAllWindows()
